In [1]:
import keras

from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Embedding, LSTM, Bidirectional, BatchNormalization, add,  Input,CuDNNGRU 
from keras.layers.core import Flatten, Reshape
from keras.layers.merge import Concatenate, concatenate, subtract, multiply
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPooling1D, AveragePooling1D, GlobalAveragePooling1D

from keras.optimizers import Adam,  RMSprop

import os
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF





os.environ["CUDA_VISIBLE_DEVICES"] = "1"

config = tf.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess = tf.Session(config=config)

KTF.set_session(sess)

Using TensorFlow backend.
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/xhh/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/xhh/anaconda3/envs/py36/lib/p

In [2]:
def build_model():
    seq_input1 = Input(shape=(seq_size, dim), name='seq1')
    seq_input2 = Input(shape=(seq_size, dim), name='seq2')
    l1=Conv1D(hidden_dim, 3)
    r1=Bidirectional(CuDNNGRU(hidden_dim, return_sequences=True))
    l2=Conv1D(hidden_dim, 3)
    r2=Bidirectional(CuDNNGRU(hidden_dim, return_sequences=True))
    l3=Conv1D(hidden_dim, 3)
    r3=Bidirectional(CuDNNGRU(hidden_dim, return_sequences=True))
    l4=Conv1D(hidden_dim, 3)
    r4=Bidirectional(CuDNNGRU(hidden_dim, return_sequences=True))
    l5=Conv1D(hidden_dim, 3)
    r5=Bidirectional(CuDNNGRU(hidden_dim, return_sequences=True))
    l6=Conv1D(hidden_dim, 3)
    s1=MaxPooling1D(3)(l1(seq_input1))
    s1=concatenate([r1(s1), s1])
    s1=MaxPooling1D(3)(l2(s1))
    s1=concatenate([r2(s1), s1])
    s1=MaxPooling1D(3)(l3(s1))
    s1=concatenate([r3(s1), s1])
    s1=MaxPooling1D(3)(l4(s1))
    s1=concatenate([r4(s1), s1])
    s1=MaxPooling1D(3)(l5(s1))
    s1=concatenate([r5(s1), s1])
    s1=l6(s1)
    s1=GlobalAveragePooling1D()(s1)
    s2=MaxPooling1D(3)(l1(seq_input2))
    s2=concatenate([r1(s2), s2])
    s2=MaxPooling1D(3)(l2(s2))
    s2=concatenate([r2(s2), s2])
    s2=MaxPooling1D(3)(l3(s2))
    s2=concatenate([r3(s2), s2])
    s2=MaxPooling1D(3)(l4(s2))
    s2=concatenate([r4(s2), s2])
    s2=MaxPooling1D(3)(l5(s2))
    s2=concatenate([r5(s2), s2])
    s2=l6(s2)
    s2=GlobalAveragePooling1D()(s2)
    merge_text = multiply([s1, s2])
    x = Dense(100, activation='linear')(merge_text)
    x = keras.layers.LeakyReLU(alpha=0.3)(x)
    x = Dense(int((hidden_dim+7)/2), activation='linear')(x)
    x = keras.layers.LeakyReLU(alpha=0.3)(x)
    main_output = Dense(2, activation='softmax')(x)
    merge_model = Model(inputs=[seq_input1, seq_input2], outputs=[main_output])
    return merge_model

In [3]:
import numpy as np
alphabet = np.array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
                     'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'])

def label_sequence(line, MAX_SEQ_LEN, smi_ch_ind):
	X = np.zeros(MAX_SEQ_LEN)

	for i, ch in enumerate(line[:MAX_SEQ_LEN]):
		X[i] = smi_ch_ind[ch]

	return X #.tolist()

def letter_one_hot(aa):
    one_hot = np.zeros(20)
    for idx, letter in enumerate(alphabet):
        if aa == letter:
            one_hot[idx] = 1
            return one_hot


# Convert an entire protein to one-hot representation.
def protein_one_hot(protein_sequence, MAX_SEQ_LEN):
    #  Remove non-specific AA codes (very few are actually present in this dataset)
    protein_sequence = protein_sequence.replace('B', '')
    protein_sequence = protein_sequence.replace('J', '')
    protein_sequence = protein_sequence.replace('O', '')
    protein_sequence = protein_sequence.replace('U', '')
    protein_sequence = protein_sequence.replace('X', '')
    protein_sequence = protein_sequence.replace('Z', '')
    one_hot_seq = np.zeros( (MAX_SEQ_LEN, 20))
    for idx, aa in enumerate(protein_sequence[:MAX_SEQ_LEN]):
        one_hot_seq[idx, :] = letter_one_hot(aa)
    return one_hot_seq


In [4]:
from six.moves import cPickle as pickle #for performance

 
def save_dict(di_, filename_):
    with open(filename_, 'wb') as f:
        pickle.dump(di_, f)

def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pickle.load(f)
    return ret_di
seq_size = 2000
protein2seq = load_dict('yeast_data/protein2seq_dicts.pkl')
protein2onehot = {}
for key, value in protein2seq.items():
    protein2onehot[key] =  protein_one_hot(value, seq_size) 

In [6]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score

n_splits = 5
TPRs =  np.zeros(n_splits)
FPRs = np.zeros(n_splits)
Precs = np.zeros(n_splits)
ACCs = np.zeros(n_splits)
F1s = np.zeros(n_splits)
MCCs = np.zeros(n_splits)
AUCs = np.zeros(n_splits)
count = 0


#copy below
num_hit = 0.
num_total = 0.
num_pos = 0.
num_true_pos = 0.
num_false_pos = 0.
num_true_neg = 0.
num_false_neg = 0.
rep = 0
class_map = {'0':1,'1':0}    

    
for split in range(0,5):
    train_pairs_file = 'yeast_data/train'+str(rep)+'-'+str(split)
    test_pairs_file = 'yeast_data/test'+str(rep)+'-'+str(split)
    valid_pairs_file = 'yeast_data/valid'+str(rep)+'-'+str(split)
    
    with open(train_pairs_file) as infile:
        train_ppis = infile.readlines()
        
    with open(test_pairs_file) as infile:
        test_ppis = infile.readlines()
        
    with open(valid_pairs_file) as infile:
        valid_ppis = infile.readlines()
    
    train_class_labels = np.zeros((len(train_ppis), 2))
    train_seq1 = np.zeros((len(train_ppis), seq_size, 20 ))
    train_seq2 = np.zeros((len(train_ppis), seq_size, 20 ))
    for i, line in  enumerate(train_ppis) :     
        p1, p2, label = line.rstrip().split('\t') 
        train_class_labels[i][class_map[label]] = 1
        train_seq1[i] = protein2onehot[p1]
        train_seq2[i] = protein2onehot[p1]
        
        
#     valid_class_labels = np.zeros((len(valid_ppis), 2))
#     valid_seq1 = np.zeros((len(valid_ppis), seq_size, 20 ))
#     valid_seq2 = np.zeros((len(valid_ppis), seq_size, 20 ))
#     for i, line in  enumerate(valid_ppis) :     
#         p1, p2, label = line.rstrip().split('\t') 
#         valid_class_labels[i][class_map[label]] = 1
#         valid_seq1[i] = protein2onehot[p1]
#         valid_seq2[i] = protein2onehot[p1]
        
    
    test_class_labels = np.zeros((len(test_ppis), 2))
    test_seq1 = np.zeros((len(test_ppis), seq_size, 20 ))
    test_seq2 = np.zeros((len(test_ppis), seq_size, 20 ))
    for i, line in  enumerate(test_ppis) :     
        p1, p2, label = line.rstrip().split('\t') 
        test_class_labels[i][class_map[label]] = 1
        test_seq1[i] = protein2onehot[p1]
        test_seq2[i] = protein2onehot[p1]
    
    batch_size1 = 256
    adam = Adam(lr=0.001, amsgrad=True, epsilon=1e-6)
    rms = RMSprop(lr=0.001)
    
    dim = 20
    hidden_dim = 50
    merge_model = None
    merge_model = build_model()
    adam = Adam(lr=0.001, amsgrad=True)
    rms = RMSprop(lr=0.001)

    merge_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    
    merge_model.fit([ train_seq1, train_seq2], train_class_labels, batch_size=batch_size1, epochs=10)
    #result1 = merge_model.evaluate([seq_tensor1[test], seq_tensor2[test]], class_labels[test])
    pred = merge_model.predict([test_seq1, test_seq2])
    y_test = np.zeros((test_class_labels.shape[0],1))
    y_pred = np.zeros((test_class_labels.shape[0],1))
    for i in range( test_class_labels.shape[0]):
        y_test[i] = np.argmax(test_class_labels[i])
        y_pred[i] = np.argmax(pred[i])
        num_total += 1
        if np.argmax(test_class_labels[i]) == np.argmax(pred[i]):
            num_hit += 1
        if train_class_labels[i][0] > 0.:
            num_pos += 1.
            if pred[i][0] > pred[i][1]:
                num_true_pos += 1
            else:
                num_false_neg += 1
        else:
            if pred[i][0] > pred[i][1]:
                num_false_pos += 1
            else:
                num_true_neg += 1
    accuracy = num_hit / num_total
    prec = num_true_pos / (num_true_pos + num_false_pos)
    recall = num_true_pos / num_pos
    spec = num_true_neg / (num_true_neg + num_false_neg)
    f1 = 2. * prec * recall / (prec + recall)
    mcc = (num_true_pos * num_true_neg - num_false_pos * num_false_neg) / ((num_true_pos + num_true_neg) * (num_true_pos + num_false_neg) * (num_false_pos + num_true_neg) * (num_false_pos + num_false_neg)) ** 0.5
    print (accuracy, prec, recall, spec, f1, mcc)
    
     
    auc = metrics.roc_auc_score(y_test, y_pred_prob) 
    f1 = f1_score(y_test, y_pred)
    pre = precision_score(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)

    precision, recall, _thresholds = metrics.precision_recall_curve(y_test, y_pred_prob)
    pr_auc = metrics.auc(recall, precision)
    mcc = matthews_corrcoef(y_test, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    total=tn+fp+fn+tp
    sen = float(tp)/float(tp+fn)
    sps = float(tn)/float((tn+fp))

    tpr = float(tp)/float(tp+fn)
    fpr = float(fp)/float((tn+fp))
    print('--------------------------\n')
    print ('AUC: %f' % auc)
    print ('ACC: %f' % acc) 
    # print("PRAUC: %f" % pr_auc)
    print ('MCC : %f' % mcc)
    # print ('SEN: %f' % sen)
    # print ('SEP: %f' % sps)
    print('TPR:%f'%tpr)
    print('FPR:%f'%fpr)
    print('Pre:%f'%pre)
    print('F1:%f'%f1)
    print('--------------------------\n')
    TPRs[count] = tpr
    FPRs[count] = fpr
    Precs[count] =pre
    ACCs[count] =acc
    F1s[count] =f1
    MCCs[count] =mcc
    AUCs[count] =auc
    count += 1

accuracy = num_hit / num_total
prec = num_true_pos / (num_true_pos + num_false_pos)
recall = num_true_pos / num_pos
spec = num_true_neg / (num_true_neg + num_false_neg)
f1 = 2. * prec * recall / (prec + recall)
mcc = (num_true_pos * num_true_neg - num_false_pos * num_false_neg) / ((num_true_pos + num_true_neg) * (num_true_pos + num_false_neg) * (num_false_pos + num_true_neg) * (num_false_pos + num_false_neg)) ** 0.5
print (accuracy, prec, recall, f1)
 
    
         
    

Epoch 1/10
7160/7160 [==============================] - 18s 3ms/step - loss: 0.6728 - acc: 0.6214
Epoch 2/10
7160/7160 [==============================] - 13s 2ms/step - loss: 0.6614 - acc: 0.6258
Epoch 3/10
7160/7160 [==============================] - 13s 2ms/step - loss: 0.6618 - acc: 0.6258
Epoch 4/10
7160/7160 [==============================] - 14s 2ms/step - loss: 0.6605 - acc: 0.6258
Epoch 5/10
7160/7160 [==============================] - 14s 2ms/step - loss: 0.6608 - acc: 0.6258
Epoch 6/10
7160/7160 [==============================] - 14s 2ms/step - loss: 0.6606 - acc: 0.6258
Epoch 7/10
7160/7160 [==============================] - 15s 2ms/step - loss: 0.6617 - acc: 0.6258
Epoch 8/10
7160/7160 [==============================] - 14s 2ms/step - loss: 0.6609 - acc: 0.6258
Epoch 9/10
7160/7160 [==============================] - 14s 2ms/step - loss: 0.6607 - acc: 0.6258
Epoch 10/10
7160/7160 [==============================] - 14s 2ms/step - loss: 0.6595 - acc: 0.6258


ZeroDivisionError: float division by zero

In [ ]:
y_test